In [1]:
import pandas as pd
import os

print("Current working directory:", os.getcwd())

# Load data from the input folder
enrollment_data = pd.read_csv("../data/input/CPSC_Enrollment_2015_01/CPSC_Enrollment_Info_2015_01.csv")
service_area_data = pd.read_csv("../data/input/MA_Cnty_SA_2015_01/MA_Cnty_SA_2015_01.csv")
contract_data = pd.read_csv("../data/input/CPSC_Enrollment_2015_01/CPSC_Contract_Info_2015_01.csv", encoding='latin1')
pd.set_option('display.max_columns', None)

#rename Contract Number to Contract ID , SSA State county code to SSA, and FIPS state county code to FIPS in enrollment data
enrollment_data.rename(columns={'Contract Number': 'Contract ID', 'SSA State County Code': 'SSA', 'FIPS State County Code': 'FIPS'}, inplace=True)

 # merge enrollment data with contract data on 'Contract ID' and 'Plan ID'
merged_data = pd.merge(enrollment_data, contract_data, on=['Contract ID', 'Plan ID'], how='left')

# #show me the merged data head
print(merged_data.head())



Current working directory: /Users/ryanscholte/Desktop/GitHub/HW1/submission1


/var/folders/mn/l2nrwsxn24g6ywz6ygh2fxp40000gn/T/ipykernel_9438/1831349687.py:8: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  service_area_data = pd.read_csv("../data/input/MA_Cnty_SA_2015_01/MA_Cnty_SA_2015_01.csv")


  Contract ID  Plan ID    SSA  FIPS State County Enrollment  \
0       E0654      801  58160   NaN   NaN    NaN          *   
1       E0654      801  61000   NaN   NaN    NaN          *   
2       E0654      801  96001   NaN   NaN    NaN          *   
3       E0654      801   2275   NaN   NaN    NaN          *   
4       E0654      801   2198   NaN   NaN    NaN          *   

                         Organization Type  \
0  Employer/Union Only Direct Contract PDP   
1  Employer/Union Only Direct Contract PDP   
2  Employer/Union Only Direct Contract PDP   
3  Employer/Union Only Direct Contract PDP   
4  Employer/Union Only Direct Contract PDP   

                                 Plan Type Offers Part D SNP Plan EGHP  \
0  Employer/Union Only Direct Contract PDP           Yes       No  Yes   
1  Employer/Union Only Direct Contract PDP           Yes       No  Yes   
2  Employer/Union Only Direct Contract PDP           Yes       No  Yes   
3  Employer/Union Only Direct Contract PDP      

In [16]:
# fill in missing FIPS values with known FIPS values matching the county and state found elsewhere in the dataset
merged_data['FIPS'] = merged_data['FIPS'].fillna(merged_data.groupby(['State', 'County'])['FIPS'].transform('first'))
# fill in missing 'Offers Part D', 'SNP Plan', 'EGHP', values with known values matching from the Plan Type found elsewhere in the dataset
merged_data['Offers Part D'] = merged_data['Offers Part D'].fillna(merged_data.groupby(['Plan Type'])['Offers Part D'].transform('first'))
merged_data['SNP Plan'] = merged_data['SNP Plan'].fillna(merged_data.groupby(['Plan Type'])['SNP Plan'].transform('first'))
merged_data['EGHP'] = merged_data['EGHP'].fillna(merged_data.groupby(['Plan Type'])['EGHP'].transform('first'))


In [17]:
# Count the number of plans for each plan type and display results largest down to smallest
plan_counts = merged_data['Plan Type'].value_counts().reset_index()
plan_counts.style.format({'Plan Count': '{:,.0f}'}).hide(axis='index')

Plan Type,count
Medicare Prescription Drug Plan,991457
Local PPO,704993
HMO/HMOPOS,479275
Employer/Union Only Direct Contract PDP,25630
Regional PPO,17578
PFFS,13658
1876 Cost,7157
MSA,6518
Medicare-Medicaid Plan HMO/HMOPOS,4130
National PACE,1216


In [18]:
#make merged data 2 set which drops all special needs plans (SNP), employer group plans (eghp), and all “800-series”
# in "SNP Plan" column drop is says yes, in "EGHP" coloumn if says yes drop it, in "Plan ID" column between 800-900 drop it
merged_data2= merged_data[~merged_data['SNP Plan'].str.contains('Yes', na=False) &
                           ~merged_data['EGHP'].str.contains('Yes', na=False) &
                           ~merged_data['Plan ID'].between(800, 900, inclusive="left")]

plan_counts2 = merged_data2['Plan Type'].value_counts().reset_index()
plan_counts2.style.format({'Plan Count': '{:,.0f}'}).hide(axis='index')

Plan Type,count
Medicare Prescription Drug Plan,269153
HMO/HMOPOS,36588
Local PPO,16728
Regional PPO,8531
1876 Cost,6329
PFFS,4232
Medicare-Medicaid Plan HMO/HMOPOS,4130
National PACE,1216
MSA,232


In [19]:
merged_data2['Enrollment'] = pd.to_numeric(merged_data2['Enrollment'], errors='coerce')
# create table for enrollment count and average enrollment for each plan type
enrollment_summary = merged_data2.groupby('Plan Type')['Enrollment'].agg(['count', 'mean']).reset_index()
enrollment_summary.columns = ['Plan Type', 'Enrollment Count', 'Average Enrollment']
enrollment_summary = enrollment_summary.sort_values(by='Enrollment Count', ascending=False)
enrollment_summary.style.format({'Enrollment Count': '{:,.0f}', 'Average Enrollment': '{:,.0f}'}).hide(axis='index')

/var/folders/mn/l2nrwsxn24g6ywz6ygh2fxp40000gn/T/ipykernel_7656/1199884363.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_data2['Enrollment'] = pd.to_numeric(merged_data2['Enrollment'], errors='coerce')


Plan Type,Enrollment Count,Average Enrollment
Medicare Prescription Drug Plan,"60,236",312
HMO/HMOPOS,"9,115",849
Local PPO,"6,126",311
Regional PPO,"4,853",202
PFFS,"2,052",125
1876 Cost,"2,019",228
Medicare-Medicaid Plan HMO/HMOPOS,522,624
National PACE,213,140
MSA,111,108
